In [2]:
import numpy as np
from math import log10
import cv2
import os

In [ ]:
def resize_and_crop(image_path, output_path):
    img = cv2.imread(image_path)    
    height, width = img.shape[:2]

    # Get smaller edge
    if width < height:
        new_width = 256
        new_height = int(height * (256 / width))
    else:
        new_height = 256
        new_width = int(width * (256 / height))

    # Resize
    img_resized = cv2.resize(img, (new_width, new_height))

    x_start = (new_width - 256) // 2
    y_start = (new_height - 256) // 2

    img_cropped = img_resized[y_start:y_start+256, x_start:x_start+256]
    cv2.imwrite(output_path, img_cropped)

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        resize_and_crop(input_path, output_path)

In [ ]:
input_folder = "./data/raw/train"
output_folder = "./data/intermediate/train"
process_folder(input_folder, output_folder)

input_folder = "./data/raw/valid"
output_folder = "./data/intermediate/valid"
process_folder(input_folder, output_folder)

In [4]:
def SNE(img1: np.ndarray, img2: np.ndarray):
    img1_flatten, img2_flatten = img1.flatten(), img2.flatten()
    result = 0
    for pix1, pix2 in zip(img1_flatten, img2_flatten):
        result += pow(pix1-pix2, 2)
    return result

def PSNR(img1: np.ndarray, img2: np.ndarray, MAX=255):
    MSE = SNE(img1, img2) / img1.size
    return 10 * log10(pow(MAX, 2)/MSE)

def SSIM(img1: np.ndarray, img2: np.ndarray, L=255, k1=0.01, k2=0.03):
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    var1 = np.var(img1)
    var2 = np.var(img2)
    cov = np.cov(img1, img2)
    c1 = (k1 * L) ** 2
    c2 = (k2 * L) ** 2
    return (2 * mean1 * mean2 + c1) * (2 * cov + c2) / ((pow(mean1, 2) * pow(mean2, 2) + c1) * (var1 * var2 + c2))

def LPIPS(img1: np.ndarray, img2: np.ndarray): ...
    # https://pypi.org/project/lpips/  